In [1]:
import pandas as pd

data = pd.read_csv('../labeled.csv', header=0)
data.head()

,id,white_elo,black_elo,moves,marks
0,787zsVup,1638.0,1851.0,e2e4 c7c6 f2f4 d7d5 e4d5 c6d5 g1f3 b8c6 d2d3 g...,"47,53"
1,F8M8OS71,1760.0,1823.0,e2e4 c7c5 g1f3 d7d6 c2c3 g8f6 d2d3 a7a6 b1d2 e...,"52,56"
2,MQSyb3KW,1877.0,1909.0,e2e4 c7c5 c2c3 d7d6 d2d4 c5d4 c3d4 b7b6 b1c3 c...,"126,130"
3,4MWQCxQ6,1741.0,1625.0,e2e4 e7e5 d2d3 b8c6 g1f3 f8c5 f1e2 g8f6 e1g1 e...,"31,35"
4,e9AY2m5j,1766.0,1733.0,e2e4 c7c5 f1c4 e7e6 g1f3 d7d5 e4d5 e6d5 c4b5 b...,"49,55"


In [2]:
games_series = data['moves'].str.split(' ')
games_series.head()

0    [e2e4, c7c6, f2f4, d7d5, e4d5, c6d5, g1f3, b8c...
1    [e2e4, c7c5, g1f3, d7d6, c2c3, g8f6, d2d3, a7a...
2    [e2e4, c7c5, c2c3, d7d6, d2d4, c5d4, c3d4, b7b...
3    [e2e4, e7e5, d2d3, b8c6, g1f3, f8c5, f1e2, g8f...
4    [e2e4, c7c5, f1c4, e7e6, g1f3, d7d5, e4d5, e6d...
Name: moves, dtype: object

In [3]:
import torch
import torch.nn as nn

DTYPE = torch.float64

class Board2Vec(nn.Module):
    def __init__(self, input_dim: int, output_dim: int):
        super(Board2Vec, self).__init__()

        # Эмбеддинги для центральных слов
        hidden_dim = 512
        layers = [
            nn.Linear(input_dim, hidden_dim, dtype=DTYPE),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim, dtype=DTYPE)
        ]
        self.target_embegging = nn.Sequential(*layers)
        
        # Эмбеддинги для контекста
        hidden_dim = 512
        layers = [
            nn.Linear(input_dim, hidden_dim, dtype=DTYPE),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim, dtype=DTYPE)
        ]
        self.context_embedding = nn.Sequential(*layers)

    def forward(self, target, context):
        # Получаем эмбеддинги для целевого и контекстного слов
        target_embed = self.target_embegging(target)
        context_embed = self.context_embedding(context)

        # Вычисляем скалярное произведение между эмбеддингами
        scores = torch.mul(target_embed, context_embed).sum(dim=1)
        log_sigmoid = torch.nn.functional.logsigmoid(scores)
        return -log_sigmoid.mean()


In [4]:
import chess
import random
from typing import List
from collections import deque

def encode_board(board: chess.Board):
    # Инициализируем пустой вектор
    vector = []
    
    # 1. Кодируем состояние доски (64 значения)
    for square in chess.SQUARES:  # chess.SQUARES — это список всех клеток от 0 до 63
        piece = board.piece_at(square)  # Получаем фигуру на клетке
        if piece is None:
            vector.append(0)  # Пустая клетка
        else:
            # Преобразуем фигуру в число: 1-6 для белых, -1-(-6) для черных
            value = piece.piece_type
            if piece.color == chess.BLACK:
                value = -value
            vector.append(value)
    
    # 2. Добавляем права на рокировку (4 бита)
    castling_rights = [
        board.has_kingside_castling_rights(chess.WHITE),  # Короткая рокировка белых
        board.has_queenside_castling_rights(chess.WHITE), # Длинная рокировка белых
        board.has_kingside_castling_rights(chess.BLACK),  # Короткая рокировка черных
        board.has_queenside_castling_rights(chess.BLACK)  # Длинная рокировка черных
    ]
    vector.extend([int(right) for right in castling_rights])
    
    # 3. Добавляем возможность взятия на проходе (1 значение)
    en_passant_square = board.ep_square  # Индекс клетки для взятия на проходе
    if en_passant_square is not None:
        vector.append(en_passant_square)
    else:
        vector.append(-1)  # Если взятие на проходе невозможно
    
    # 4. Добавляем текущего игрока (1 бит)
    current_player = int(board.turn)  # 1 для белых, 0 для черных
    vector.append(current_player)
    
    return vector

class TargetContextBoardsLoader:
    def __init__(self, games: List[str], window_size: int, epoch_size: int):
        self.games = games
        self.length = len(self.games)
        self.window_size = window_size
        self.epoch_size = epoch_size
        self.left = epoch_size
        self.set_game()
        self.prepare_game()

    def set_game(self):
        self.cur_game = random.randint(0, self.length - 1)
        self.left -= 1

    def prepare_game(self):
        self.boards = []
        prev_board = chess.Board()
        game = self.games[self.cur_game]
        for move in game:
            prev_board.push(chess.Move.from_uci(move))
            encoded = encode_board(prev_board)
            self.boards.append(encoded)
        self.cur_move = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.cur_move >= len(self.boards):
            self.set_game()
            if self.left < 0:
                self.left = self.epoch_size
                raise StopIteration
            
            self.prepare_game()

        target = self.boards[self.cur_move]

        if self.cur_move == 0:
            self.left_context = deque()
            self.right_context = deque()
            for j in range(1, min(len(self.boards), 1 + self.window_size)):
                self.right_context.append(self.boards[j])
            self.cur_move += 1

            context = list(self.left_context) + list(self.right_context)
            return torch.tensor([target] * len(context), dtype=DTYPE), torch.tensor(context, dtype=DTYPE)
    
        if len(self.left_context) > 0:
            self.left_context.popleft()
        self.left_context.append(self.boards[self.cur_move - 1])

        if self.window_size + self.cur_move < len(self.boards):
            self.right_context.append(self.boards[self.window_size + self.cur_move])
        self.right_context.popleft()
        self.cur_move += 1

        context = list(self.left_context) + list(self.right_context)
        return torch.tensor([target] * len(context), dtype=DTYPE), torch.tensor(context, dtype=DTYPE)

In [7]:
dataloader = TargetContextBoardsLoader(games_series, window_size=5, epoch_size=10)

In [9]:
import torch.optim as optim

# Модель
input_dim = 70
output_dim = 256
model = Board2Vec(input_dim, output_dim)

# Гиперпараметры
learning_rate = 0.00001
num_epochs = 50

# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Цикл обучения
for epoch in range(num_epochs):
    total_loss = 0
    for target, context in dataloader:
        optimizer.zero_grad()
        loss = model(target, context)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss*100:.4f}")

Epoch [1/50], Loss: 3756.2173
Epoch [2/50], Loss: 166.6083
Epoch [3/50], Loss: 58.5451
Epoch [4/50], Loss: 10.9591
Epoch [5/50], Loss: 57.6123
Epoch [6/50], Loss: 17.5653
Epoch [7/50], Loss: 13.7337
Epoch [8/50], Loss: 1.1947
Epoch [9/50], Loss: 4.9962
Epoch [10/50], Loss: 0.3219
Epoch [11/50], Loss: 0.0128
Epoch [12/50], Loss: 2.9353
Epoch [13/50], Loss: 0.3016
Epoch [14/50], Loss: 0.9994
Epoch [15/50], Loss: 0.0743
Epoch [16/50], Loss: 0.3459
Epoch [17/50], Loss: 0.1443
Epoch [18/50], Loss: 0.0142
Epoch [19/50], Loss: 0.1119
Epoch [20/50], Loss: 0.2550
Epoch [21/50], Loss: 0.0641
Epoch [22/50], Loss: 0.4187
Epoch [23/50], Loss: 0.0043
Epoch [24/50], Loss: 0.0286
Epoch [25/50], Loss: 0.0014
Epoch [26/50], Loss: 0.0000
Epoch [27/50], Loss: 0.0020
Epoch [28/50], Loss: 0.0002
Epoch [29/50], Loss: 0.0019
Epoch [30/50], Loss: 0.0241
Epoch [31/50], Loss: 0.0028


KeyboardInterrupt: 